In [2]:
import os
import cv2
import torch
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
from torch import nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'torch._prims_common'

In [1]:
def extract_frames(video_path, num_frames=20):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames).astype(int)
    frames = []

    for idx in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if idx in frame_indices:
            frame = cv2.resize(frame, (112, 112))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    cap.release()
    return frames if len(frames) == num_frames else None

In [ ]:
def prepare_dataset(root_dir):
    data = []
    for label in ['real', 'fake']:
        folder = os.path.join(root_dir, label)
        for video in os.listdir(folder):
            if video.endswith(".mp4"):
                path = os.path.join(folder, video)
                data.append((path, 1 if label == 'real' else 0))
    return data



In [ ]:
class VideoDataset(Dataset):
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        video_path, label = self.data_list[idx]
        frames = extract_frames(video_path)
        if frames is None:
            return self.__getitem__((idx + 1) % len(self.data_list))  # try next video

        if self.transform:
            frames = [self.transform(Image.fromarray(frame)) for frame in frames]

        frames_tensor = torch.stack(frames)
        return frames_tensor, label



In [ ]:
class DeepfakeModel(nn.Module):
    def __init__(self, num_classes, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(DeepfakeModel, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size, seq_length, 2048)
        x_lstm, _ = self.lstm(x, None)
        return fmap, self.dp(self.linear1(torch.mean(x_lstm, dim=1)))

In [ ]:
def train_model(train_loader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        _, outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    return epoch_loss, accuracy

In [ ]:
def main():
    root_dir = "dataset"  
    data_list = prepare_dataset(root_dir)
    train_list, test_list = train_test_split(data_list, test_size=0.2, random_state=42)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])

    train_dataset = VideoDataset(train_list, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DeepfakeModel(num_classes=2).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(5):
        loss, acc = train_model(train_loader, model, criterion, optimizer, device)
        print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {acc:.2f}%")

if __name__ == "__main__":
    main()


In [ ]:
def preprocess_video(video_path, frame_count=100):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(total_frames // frame_count, 1)

    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(frame)
        
        if face_locations:
            top, right, bottom, left = face_locations[0]
            face = frame[top:bottom, left:right]
        else:
            face = frame  
        
        face = cv2.resize(face, (224, 224))

        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])
        frames.append(transform(face))

    cap.release()
    if not frames:
        return None  
    
    frames = torch.stack(frames)  
    return frames.unsqueeze(0)


def predict_video(model, device, video_path):
    frames = preprocess_video(video_path)
    if frames is None:
        print("No frames extracted. Check video format.")
        return

    frames = frames.to(device)  
    print(f"Input shape to model: {frames.shape}")  

    with torch.no_grad():
        _, output = model(frames)  
        probs = torch.softmax(output, dim=1)  
        confidence, predicted_class = torch.max(probs, dim=1)  

    label = "FAKE" if predicted_class.item() == 1 else "REAL"
    print(f"Prediction: {label} (Confidence: {confidence.item():.4f})")
    return label